In [6]:
import re
import time

from cltk.corpus.utils.formatter import assemble_phi5_works_filepaths
from cltk.corpus.utils.formatter import phi5_plaintext_cleanup
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

In [2]:
def works_texts_list():
    fps = assemble_phi5_works_filepaths()
    curly_comp = re.compile(r'{.+?}')
    _list = []
    for fp in fps:
        with open(fp) as fo:
            fr = fo.read()
        text = phi5_plaintext_cleanup(fr, rm_punctuation=True, rm_periods=True)
        text = curly_comp.sub('', text)
        _list.append(text)
    return _list

In [3]:
t0 = time.time()
my_phrases = works_texts_list()
print('Time to build list of texts: {}'.format(time.time() - t0))

In [5]:
t1 = time.time()
vectorizer = TfidfVectorizer(min_df=1)
my_features = vectorizer.fit_transform(my_phrases)
feature_words = vectorizer.get_feature_names()
print('Time to create vectorizer: {}'.format(time.time() - t1))

In [7]:
t2 = time.time()
D = {}
for doc_index, weights in enumerate(my_features.A):
    D[doc_index] = pd.Series(weights, index=feature_words)
df = pd.DataFrame(D)

df_t = df.transpose()
print('Time to load scores into pandas: {}'.format(time.time() - t2))

Time to load scores into pandas: 201.71255683898926


In [8]:
df_t.shape

(836, 318869)